# 📊 PDF 데이터시트 전처리 도구

**데이터시트 PDF를 분석하여 유용한 정보를 추출하고 JSON 형태로 변환합니다.**

---

## 🚀 시작하기 전에

이 노트북은 PDF 데이터시트를 처리하여 나중에 챗봇에서 사용할 수 있는 JSON 파일을 생성합니다.
분할 방식으로 PDF를 처리하여 더 효율적으로 정보를 추출합니다.

## 📋 사용 방법
아래 셀들을 **순서대로 실행**만 하면 됩니다. (각 셀의 ▶️ 버튼을 클릭)

---

In [ ]:
#@title ✅ 1단계: Google Drive 연동 { display-mode: "form" }
#@markdown Google Drive를 연동하여 PDF 파일을 불러오고 처리된 JSON 파일과 벡터 스토어를 저장합니다.

from google.colab import drive
import os
from pathlib import Path

print("🔄 Google Drive 마운트 중...")
drive.mount('/content/drive')

# 폴더 경로 설정
pdf_folder = Path('/content/drive/MyDrive/datasheets')
pre_json_folder = Path('/content/drive/MyDrive/prep_json/pre_json')
result_json_folder = Path('/content/drive/MyDrive/prep_json')
vectorstore_folder = Path('/content/drive/MyDrive/vectorstore')

# 폴더가 없으면 생성
pdf_folder.mkdir(parents=True, exist_ok=True)
pre_json_folder.mkdir(parents=True, exist_ok=True)
result_json_folder.mkdir(parents=True, exist_ok=True)
vectorstore_folder.mkdir(parents=True, exist_ok=True)

print(f"\n✅ Google Drive 연동 완료!")
print(f"📁 PDF 폴더: {pdf_folder}")
print(f"📁 중간 JSON 폴더: {pre_json_folder}")
print(f"📁 최종 JSON 폴더: {result_json_folder}")
print(f"📁 벡터 스토어 폴더: {vectorstore_folder}")

# PDF 파일 목록 확인
pdf_files = list(pdf_folder.glob("*.pdf"))
print(f"\n�� 발견된 PDF 파일: {len(pdf_files)}개")
for pdf in pdf_files[:5]:  # 처음 5개만 표시
    print(f" - {pdf.name}")
    
if len(pdf_files) > 5:
    print(f" ... 외 {len(pdf_files) - 5}개")

if not pdf_files:
    print(f"\n⚠️ PDF 파일이 없습니다. '{pdf_folder}' 폴더에 PDF 파일을 업로드해주세요.")

In [ ]:
#@title 📥 2단계: 필요한 라이브러리 설치 { display-mode: "form" }
#@markdown 필요한 라이브러리들을 설치합니다.

import os
from pathlib import Path
import json

print("📥 필요한 라이브러리 설치 중...")

# GitHub 저장소 클론
!git clone https://github.com/doyoung42/chipchat.git
%cd chipchat/prep

# requirements.txt 설치
!pip install -r requirements.txt -q

# 현재 디렉토리 설정
import sys
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

# 설정 파일 로드
models_path = Path('misc/models.json')
param_path = Path('misc/param.json')

with open(models_path, 'r') as f:
    models = json.load(f)
with open(param_path, 'r') as f:
    params = json.load(f)

print("✅ 라이브러리 설치 완료!")
print("\n📋 사용 가능한 모델 목록:")
print("\nOpenAI 모델:")
for model in models["openai_models"]:
    print(f"• {model}")
print("\nClaude 모델:")
for model in models["claude_models"]:
    print(f"• {model}")

print("\n📋 현재 PDF 처리 설정:")
print(f"• 페이지당 청크 수: {params['pdf_processing']['pages_per_chunk']}")
print(f"• 출력 형식:")
print(f"  - 필터링된 PDF 저장: {params['pdf_processing']['output_formats']['save_filtered_pdf']}")
print(f"  - 카테고리별 시맨틱 청킹: 활성화됨")
print(f"  - 페이지 요약 및 카테고리 분류: 활성화됨")

In [ ]:
#@title 🔑 3단계: API 키 설정 { display-mode: "form" }
#@markdown API 키를 입력하고 사용할 LLM 모델을 선택합니다.

import os
import json
from pathlib import Path

# API 키 입력
openai_api_key = "" #@param {type:"string"}
claude_api_key = "" #@param {type:"string"}
hf_token = "" #@param {type:"string"}

# LLM 모델 선택
llm_model = "claude-3-sonnet-20240229" #@param ["claude-3-5-sonnet-20241022", "claude-3-opus-20240229", "claude-3-sonnet-20240229", "claude-3-haiku-20240307", "gpt-3.5-turbo-0125", "gpt-4o-mini-2024-07-18"] {type:"string"}

# API 키 유효성 검사
if not openai_api_key and not claude_api_key and not hf_token:
    print("⚠️ API 키가 입력되지 않았습니다.")
    print("💡 최소 하나 이상의 API 키를 입력해주세요.")
    print("• OpenAI API 키: https://platform.openai.com/api-keys")
    print("• Claude API 키: https://console.anthropic.com/keys")
    print("• HuggingFace 토큰: https://huggingface.co/settings/tokens")
else:
    # key.json 파일에 저장
    key_path = Path('misc/key.json')
    key_path.parent.mkdir(exist_ok=True)
    
    # 모델 타입 결정
    if llm_model.startswith("claude"):
        provider = "claude"
    else:
        provider = "openai"
    
    with open(key_path, 'w') as f:
        json.dump({
            'openai_api_key': openai_api_key,
            'anthropic_api_key': claude_api_key,
            'huggingface_api_key': hf_token,
            'selected_models': {
                provider: llm_model
            }
        }, f, indent=2)
    
    print("✅ API 키와 선택된 모델이 key.json 파일에 저장되었습니다.")

In [ ]:
#@title 📊 4단계: PDF 파일 처리 설정 { display-mode: "form" }
#@markdown PDF 처리 설정을 구성합니다.

import json
from pathlib import Path

# param.json 파일 로드
param_path = Path('misc/param.json')
with open(param_path, 'r') as f:
    params = json.load(f)

# 현재 설정값 표시
print("📋 현재 PDF 처리 설정:")
print(f"• 페이지당 청크 수: {params['pdf_processing']['pages_per_chunk']}")
print(f"• 출력 형식:")
print(f"  - 필터링된 PDF 저장: {params['pdf_processing']['output_formats']['save_filtered_pdf']}")
print(f"  - 카테고리별 시맨틱 청킹: 활성화됨")
print(f"  - 페이지 요약 및 카테고리 분류: 활성화됨")

# 설정값 수정
pages_per_chunk = 3 #@param {type:"slider", min:1, max:10, step:1}
save_filtered_pdf = True #@param {type:"boolean"}
save_summary_only = True #@param {type:"boolean"} 
save_combined = True #@param {type:"boolean"}

# 설정값 업데이트
params["pdf_processing"]["pages_per_chunk"] = pages_per_chunk
params["pdf_processing"]["output_formats"]["save_filtered_pdf"] = save_filtered_pdf
params["pdf_processing"]["output_formats"]["save_summary_only"] = save_summary_only
params["pdf_processing"]["output_formats"]["save_combined"] = save_combined

# 폴더 경로 업데이트
params["pdf_processing"]["folders"]["pdf_folder"] = str(pdf_folder)
params["pdf_processing"]["folders"]["pre_json_folder"] = str(result_json_folder / "pre_json")
params["pdf_processing"]["folders"]["result_json_folder"] = str(result_json_folder)
params["vectorstore"]["folders"]["vectorstore_folder"] = str(vectorstore_folder)

# param.json 파일에 저장
param_path = Path('misc/param.json')
param_path.parent.mkdir(exist_ok=True)

with open(param_path, 'w') as f:
    json.dump(params, f, indent=2)

print("\n✅ PDF 처리 설정이 param.json 파일에 저장되었습니다.")
print("\n📝 설정 설명:")
print("• 페이지당 청크 수: PDF를 몇 페이지씩 묶어서 처리할지 설정합니다.")
print("• 필터링된 PDF 저장: 유용한 정보가 있는 페이지만 모아서 PDF로 저장합니다.")
print("• 카테고리별 처리: 내용을 다음 카테고리로 분류하여 처리합니다:")
print("  - 제품 요약(Product summary)")
print("  - 전기적 특성(Electrical Characteristics)")
print("  - 응용 회로 예시(Application circuits)")
print("  - 기계적 특성(Mechanical Characteristics)")
print("  - 신뢰성 및 환경 시험 조건(Reliability, Environmental conditions)")
print("  - 패키징 정보(Packaging)")

In [ ]:
#@title 📄 5단계: PDF 파일 처리 { display-mode: "form" }
#@markdown PDF 파일들을 처리합니다.

from src.pdf_processor import PDFProcessor
from src.llm_manager import LLMManager
import json
from pathlib import Path

# key.json에서 선택된 모델 정보 로드
key_path = Path('misc/key.json')
with open(key_path, 'r') as f:
    keys = json.load(f)
selected_models = keys.get('selected_models', {})

# 선택된 모델에 따라 provider 결정
if any(model.startswith('claude') for model in selected_models.values()):
    provider = "claude"
    model_name = selected_models.get('claude')
else:
    provider = "openai"
    model_name = selected_models.get('openai')

# LLM Manager 초기화
llm_manager = LLMManager(provider=provider, model_name=model_name)

# PDF Processor 초기화
processor = PDFProcessor(llm_manager)

# PDF 처리 시작
print("🔄 PDF 파일 처리 시작...")
processor.process_pdf_folder()
print("✅ PDF 파일 처리 완료!")

In [ ]:
#@title 🔍 6단계: 벡터 스토어 생성 { display-mode: "form" }
#@markdown 처리된 JSON 파일들로부터 벡터 스토어를 생성합니다.

from src.vectorstore_manager import VectorstoreManager
import json
from pathlib import Path

# Vectorstore Manager 초기화
vectorstore_manager = VectorstoreManager()

# JSON 파일 로드
json_folder = Path(default_params["pdf_processing"]["folders"]["result_json_folder"])
json_files = list(json_folder.glob("*_R1.json"))  # R1 파일만 사용

if not json_files:
    print("❌ 처리된 JSON 파일이 없습니다. 5단계를 먼저 실행해주세요.")
else:
    print(f"📄 발견된 JSON 파일: {len(json_files)}개")
    
    # JSON 데이터 로드
    json_data = []
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                json_data.append(data)
            print(f"✅ {json_file.name} 로드 완료")
        except Exception as e:
            print(f"❌ {json_file.name} 로드 실패: {str(e)}")
    
    if json_data:
        print("\n🔄 벡터 스토어 생성 중...")
        try:
            # 벡터 스토어 생성
            vectorstore = vectorstore_manager.create_vectorstore(json_data)
            
            # 벡터 스토어 저장
            vectorstore_manager.save_vectorstore(vectorstore, "datasheet_vectors")
            
            print("✅ 벡터 스토어 생성 및 저장 완료!")
            print(f"💾 저장 위치: {vectorstore_folder}/datasheet_vectors")
        except Exception as e:
            print(f"❌ 벡터 스토어 생성 실패: {str(e)}")
    else:
        print("❌ 로드된 JSON 데이터가 없습니다.")